# 🧪 LLM.int8() From Scratch: Outlier-Aware Quantization (2022)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/model-size-reduction/blob/main/chronology/llm_int8_demo.ipynb)

## 📖 Theory: Vector-wise Quantization & Outliers

LLM.int8() (Dettmers et al., 2022) was the first method to make 8-bit inference
practical for models with **>6.7 billion parameters**, where a surprising
phenomenon breaks naive INT8 quantization.

### The Emergent Outlier Problem

As LLMs scale past ~6.7B parameters, a small fraction of hidden-state feature
dimensions develop **systematic, massive magnitudes** -- values of order $10^1$-$10^2$
while the rest of the tensor sits at $10^{-1}$-$10^{-2}$.

These *outlier dimensions* appear in the **same fixed set of channels** across all
tokens and all layers. They are not random; they encode critical model structure.

### Why Naive INT8 Fails

Standard INT8 maps $[x_{\min}, x_{\max}]$ to $[-128, 127]$ with a single scale:

$$\text{scale} = \frac{\max|X|}{127}$$

If $\max|X| = 52$, then scale $\approx 0.41$, and a normal activation of $0.12$
quantizes to $\text{round}(0.12/0.41) = 0$ -- **total precision loss** for
the majority of features.

### The Mixed-Precision Decomposition

$$Y = X_{\text{int8}} W_{\text{int8}} + X_{\text{out}} W_{\text{out}}$$

1. **Identify outlier columns** in $X$: those where $\max|X_{:,j}| > \theta$ (typically $\theta = 6$).
2. **Outlier path** (~1% of features): multiply in **FP16** -- exact, no rounding error.
3. **Normal path** (~99% of features): quantize with **vector-wise INT8** scales.

### Vector-wise vs Tensor-wise Quantization

| | Tensor-wise | Vector-wise |
|---|---|---|
| Scale granularity | One scale for the whole matrix | One scale per row of $X$, per column of $W$ |
| Precision loss | High (outliers dominate scale) | Low (each scale fits its row/column) |
| Memory overhead | Negligible | One float32 per row/column |

Vector-wise scales reduce INT8 quantization error by 3-10x on normal features.

### Threshold Selection

The outlier threshold $\theta = 6.0$ was chosen empirically:
* Below 6.7B params: no systematic outliers -- standard INT8 suffices.
* Above 6.7B params: outlier dimensions emerge -- $\theta = 6$ cleanly separates them.

### Memory Savings

For a 175B-parameter model:
* FP16 baseline: $175 \times 10^9 \times 2 \approx 350$ GB
* LLM.int8(): $175 \times 10^9 \times 1 \approx 175$ GB (~**2x reduction**)

This reduced 175B inference from 5-8 A100s down to 2-3 A100s.

### Limitations
* INT8 arithmetic is slower than FP16 on some GPU architectures.
* Mixed-precision split adds implementation complexity.
* Designed for 8-bit only -- accuracy degrades at 4-bit.

---

In [ ]:
import torch
import time
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 1. Load Model & Tokenizer
model_id = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_id)
model = GPT2LMHeadModel.from_pretrained(model_id).eval()

def print_size_in_mb(tensor, name):
    size = tensor.numel() * tensor.element_size() / (1024**2)
    print(f"{name} Size: {size:.2f} MB")

weight = model.transformer.h[0].attn.c_attn.weight.data
print_size_in_mb(weight, "Original FP32 Weight")

## 🛠️ Implementation: Manual Vector-wise Quantization
Instead of using black-box libraries, we implement the core math manually to demonstrate the efficiency gains.

In [ ]:
def llm_int8_matmul(X, W, threshold=6.0):
    """
    A from-scratch simulation of outlier-aware matmul.
    """
    # 1. Identify Outlier Dimensions
    X = X.float()
    W = W.float()
    col_max = torch.max(torch.abs(X), dim=0)[0]
    outlier_idx = torch.where(col_max > threshold)[0]
    non_outlier_idx = torch.where(col_max <= threshold)[0]
    
    # 2. Extract and multiply Outliers in high precision
    X_outlier = X[:, outlier_idx]
    W_outlier = W[outlier_idx, :]
    outlier_res = torch.matmul(X_outlier, W_outlier)
    
    # 3. Quantize and multiply others in INT8 simulation
    X_non = X[:, non_outlier_idx]
    W_non = W[non_outlier_idx, :]
    
    if X_non.shape[1] > 0:
        # Vector-wise quant for X (row units)
        x_scale = torch.max(torch.abs(X_non), dim=1, keepdim=True)[0] / 127
        X_q = torch.round(X_non / (x_scale + 1e-8)).to(torch.int8)
        
        # Vector-wise quant for W (col units)
        w_scale = torch.max(torch.abs(W_non), dim=0, keepdim=True)[0] / 127
        W_q = torch.round(W_non / (w_scale + 1e-8)).to(torch.int8)
        
        # Matmul simulation (using Torch's matmul on float32 converted from int8)
        int8_res = torch.matmul(X_q.to(torch.float32), W_q.to(torch.float32))
        int8_res = int8_res * (x_scale * w_scale)
    else:
        int8_res = 0
    
    return int8_res + outlier_res

# Test with synthetic data containing outliers
X = torch.randn(16, 512)
X[:, 7] *= 20.0 # Dimension 7 is an outlier
W = torch.randn(512, 1024)

start = time.time()
fp32_res = torch.matmul(X, W)
time_fp32 = time.time() - start

start = time.time()
int8_res = llm_int8_matmul(X, W)
time_int8 = time.time() - start

mse = torch.mean((fp32_res - int8_res)**2)
print(f"Reconstruction MSE (with outliers): {mse:.8f}")
print(f"FP32 Time: {time_fp32:.4f}s")
print(f"INT8 Simulation Time: {time_int8:.4f}s")
print(f"\nWeight storage effectively reduced to 8-bit (4x compression simulation)!")

## 🔢 Worked Example with Numbers

Before the full implementation, let’s trace through the math with a tiny, hand-traceable example.

In [ ]:
import torch

print("=== LLM.int8(): Outlier problem on a 1×6 activation vector ===\n")

# 6-dim activation row: feature 3 is a massive outlier
X_row = torch.tensor([[0.12, -0.09,  0.11, 52.3, -0.08,  0.14]])  # shape [1, 6]
W_col = torch.tensor([[ 0.5], [-0.3], [ 0.7], [0.1], [-0.6], [ 0.4]])  # shape [6, 1]

print(f"Activation X (1×6): {X_row.tolist()[0]}")
print(f"  Feature 3 = 52.3  ← OUTLIER  (others ≈ 0.1)")
print(f"Weight column W (6×1): {[v[0] for v in W_col.tolist()]}")

# True dot product
# 0.12×0.5 + (−0.09)×(−0.3) + 0.11×0.7 + 52.3×0.1 + (−0.08)×(−0.6) + 0.14×0.4
# = 0.060 + 0.027 + 0.077 + 5.230 + 0.048 + 0.056
fp32_result = (X_row @ W_col).item()  # => 5.4980
print(f"\nTrue FP32 result: {fp32_result:.4f}")

# ── Naive INT8 (global scale) ───────────────────────────────────────────────
print("\n--- Naive INT8 (global scale) ---")

# Scale is dominated by the outlier 52.3 → normal values lose all precision
x_scale = X_row.abs().max() / 127          # => 52.3 / 127 = 0.4118
X_q = (X_row / x_scale).round().clamp(-127, 127).to(torch.int8)
# Each value ÷ 0.4118:  [0.29→0, -0.22→0, 0.27→0, 127.0→127, -0.19→0, 0.34→0]
# X_q                                                => [0, 0, 0, 127, 0, 0]

print(f"Scale = {X_row.abs().max():.1f}/127 = {x_scale:.4f}")
print(f"INT8 X: {X_q.tolist()[0]}")
print(f"  → Normal values (0.12) quantize to  {(0.12/x_scale.item()):.2f} → 0  (precision destroyed!)")

w_scale = W_col.abs().max() / 127          # => 0.7 / 127 = 0.0055
W_q = (W_col / w_scale).round().clamp(-127, 127).to(torch.int8)
# Each value ÷ 0.0055: [90.7→91, -54.4→-54, 127.0→127, 18.1→18, -108.9→-109, 72.6→73]
# W_q                                                => [[91], [-54], [127], [18], [-109], [73]]

# Dot product in int8 space: only the outlier term survives
# [0,0,0,127,0,0] · [91,-54,127,18,-109,73] = 127×18 = 2286
# Rescale: 2286 × 0.4118 × 0.0055
naive_result = (X_q.float() @ W_q.float() * x_scale * w_scale).item()  # => 5.1887
print(f"Naive INT8 result: {naive_result:.4f}   error = {abs(fp32_result - naive_result):.4f}")
# error => 0.3093  (the 5 normal features contributed nothing)

# ── LLM.int8(): separate outlier → FP32, normal → INT8 ────────────────────
print("\n--- LLM.int8() (outlier separated) ---")

out_idx  = [3]           # outlier features → FP32 path
norm_idx = [0,1,2,4,5]  # normal features  → INT8 path

# FP32 path: exact multiplication for the outlier
fp_part = (X_row[:, out_idx] @ W_col[out_idx, :]).item()  # => 52.3 × 0.1 = 5.2300

# INT8 path for normal features (vector-wise scale)
X_n = X_row[:, norm_idx]    # => [[0.12, -0.09, 0.11, -0.08, 0.14]]
W_n = W_col[norm_idx, :]    # => [[0.5], [-0.3], [0.7], [-0.6], [0.4]]

xs = X_n.abs().max() / 127  # => 0.14 / 127 = 0.0011   (small: fits normal values well)
ws = W_n.abs().max() / 127  # => 0.70 / 127 = 0.0055

X_nq = (X_n / xs).round().clamp(-127, 127).to(torch.int8)
# [0.12/0.0011→109, -0.09/0.0011→-82, 0.11/0.0011→100, -0.08/0.0011→-73, 0.14/0.0011→127]
# X_nq                                               => [109, -82, 100, -73, 127]

W_nq = (W_n / ws).round().clamp(-127, 127).to(torch.int8)
# [0.5/0.0055→91, -0.3/0.0055→-54, 0.7/0.0055→127, -0.6/0.0055→-109, 0.4/0.0055→73]
# W_nq                                               => [[91], [-54], [127], [-109], [73]]

# INT8 dot product then rescale:
# 109×91 + (-82)×(-54) + 100×127 + (-73)×(-109) + 127×73
# = 9919 + 4428 + 12700 + 7957 + 9271 = 44275
# × 0.0011 × 0.0055
int8_part = (X_nq.float() @ W_nq.float() * xs * ws).item()  # => 44275 × xs × ws ≈ 0.2690

combined = fp_part + int8_part  # => 5.2300 + 0.2690 = 5.4990

print(f"FP32 outlier path:  {X_row[0,3]:.1f} × {W_col[3,0]:.1f} = {fp_part:.4f}")
# => 52.3 × 0.1 = 5.2300
print(f"INT8 normal path:   {int8_part:.4f}")
# => ≈ 0.2690
print(f"Combined:           {combined:.4f}   error = {abs(fp32_result - combined):.6f}")
# => 5.4990   error ≈ 0.001000

print(f"\nNaive INT8 error : {abs(fp32_result - naive_result):.4f}")   # => 0.3093
print(f"LLM.int8() error : {abs(fp32_result - combined):.6f}  ({abs(fp32_result-naive_result)/max(abs(fp32_result-combined),1e-9):.0f}× better)")
# => ~300× better  (the outlier is handled exactly; normals fit the INT8 range perfectly)


## 📚 References

1. **Dettmers, T., Lewis, M., Belkada, Y., & Zettlemoyer, L.** (2022).  
   *LLM.int8(): 8-bit Matrix Multiplication for Transformers at Scale.* NeurIPS 2022.  
   [arXiv:2208.07339](https://arxiv.org/abs/2208.07339)

2. **Dettmers, T., Svirschevski, R., Egiazarian, V., et al.** (2023).  
   *SpQR: A Sparse-Quantized Representation for Near-Lossless LLM Weight Compression.*  
   [arXiv:2306.03078](https://arxiv.org/abs/2306.03078)

3. **Xiao, G., Lin, J., Seznec, M., Wu, H., Demouth, J., & Han, S.** (2022).  
   *SmoothQuant: Accurate and Efficient Post-Training Quantization for Large Language Models.* ICML 2023.  
   [arXiv:2211.10438](https://arxiv.org/abs/2211.10438)

4. **Zafrir, O., Boudoukh, G., Izsak, P., & Wasserblat, M.** (2019).  
   *Q8BERT: Quantized 8Bit BERT.*  
   [arXiv:1910.06188](https://arxiv.org/abs/1910.06188)


## 🧪 GPT-2 Evaluation

Apply vector-wise INT8 quantization to all 2D weight matrices of GPT-2 and compare perplexity before and after.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch, copy

model_id = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_id)
model = GPT2LMHeadModel.from_pretrained(model_id).eval()

text = "The quick brown fox jumps over the lazy dog. Transformers are powerful sequence models."
inputs = tokenizer(text, return_tensors="pt")

def perplexity(mdl, inputs):
    with torch.no_grad():
        loss = mdl(**inputs, labels=inputs["input_ids"]).loss
    return torch.exp(loss).item()

baseline_ppl = perplexity(model, inputs)
print(f"Baseline GPT-2 Perplexity: {baseline_ppl:.2f}")

# Apply vector-wise INT8 quantization to all 2D weight matrices
# (simulates the LLM.int8() weight path without outlier separation for a global view)
model_q = copy.deepcopy(model)
for name, param in model_q.named_parameters():
    if param.dim() == 2:
        W = param.data.float()
        # Vector-wise scale: one scale per output row
        row_scale = W.abs().max(dim=1, keepdim=True)[0] / 127 + 1e-8
        W_q = (W / row_scale).round().clamp(-127, 127) * row_scale
        param.data = W_q.to(param.dtype)

quant_ppl = perplexity(model_q, inputs)
print(f"LLM.int8() GPT-2 Perplexity: {quant_ppl:.2f}")
print(f"Delta:                        {quant_ppl - baseline_ppl:+.2f}")


In [ ]:
import torch, torch.nn as nn, copy

try:
    from transformers.pytorch_utils import Conv1D
except ImportError:
    from transformers.modeling_utils import Conv1D

# ── Full LLM.int8(): outlier-aware mixed-precision decomposition ─────────────
# Replaces every Conv1D layer with a module that:
#   • stores W in INT8 with per-input-row scale
#   • at forward time detects outlier activation columns (|col_max| > θ)
#   • routes outlier dims → FP32 matmul  (exact, no rounding error)
#   • routes normal  dims → INT8-simulated matmul (quantised activations + INT8 weight)

class OutlierAwareLinear(nn.Module):
    """
    Drop-in for GPT-2's Conv1D.
    Weight stored as INT8 (row-wise scale).
    Outlier input dimensions → FP32 path; normal dims → INT8 path.
    """
    def __init__(self, conv1d: Conv1D, threshold: float = 6.0):
        super().__init__()
        self.threshold = threshold
        W = conv1d.weight.data.float()                                    # [in_features, out_features]
        row_scale = W.abs().max(dim=1, keepdim=True)[0] / 127 + 1e-8    # one scale per input row
        self.register_buffer("W_int8",    (W / row_scale).round().clamp(-127, 127).to(torch.int8))
        self.register_buffer("row_scale", row_scale)
        bias = conv1d.bias
        self.register_buffer("bias_buf",  bias.data.float() if bias is not None else None)

    def forward(self, x):
        orig_shape = x.shape
        x_flat = x.reshape(-1, x.shape[-1]).float()           # [N, in_features]
        W_fp   = self.W_int8.float() * self.row_scale         # dequantize weights: [in, out]

        # Detect outlier input-feature columns across the current batch
        col_max   = x_flat.abs().max(dim=0)[0]                # max |activation| per feature
        out_mask  = col_max > self.threshold                   # True → outlier feature
        norm_mask = ~out_mask

        # FP32 path: exact multiply for high-magnitude features
        fp_part = x_flat[:, out_mask] @ W_fp[out_mask] if out_mask.any() else 0.0

        # INT8-simulated path: quantise activations, use INT8 weights
        if norm_mask.any():
            X_n     = x_flat[:, norm_mask]                    # [N, n_normal]
            W_n_fp  = W_fp[norm_mask]                         # dequantized weight slice
            # Vector-wise activation scale (one per row of X_n)
            x_scale = X_n.abs().max(dim=1, keepdim=True)[0] / 127 + 1e-8
            X_nq    = (X_n / x_scale).round().clamp(-127, 127) * x_scale   # quantize→dequantize
            int8_part = X_nq @ W_n_fp
        else:
            int8_part = 0.0

        out = fp_part + int8_part
        if self.bias_buf is not None:
            out = out + self.bias_buf
        return out.reshape(*orig_shape[:-1], -1).to(x.dtype)


def replace_with_outlier_aware(mdl, threshold=6.0):
    """Collect all Conv1D modules then replace them (safe: no mid-iteration mutation)."""
    replacements = []
    for parent in mdl.modules():
        for child_name, child in parent.named_children():
            if isinstance(child, Conv1D):
                replacements.append((parent, child_name, child))
    for parent, child_name, child in replacements:
        setattr(parent, child_name, OutlierAwareLinear(child, threshold=threshold))
    return mdl


model_q2 = replace_with_outlier_aware(copy.deepcopy(model), threshold=6.0)

# Show how many outlier dimensions appear in the first layer's input
with torch.no_grad():
    emb = model.transformer.wte(inputs["input_ids"])          # [1, seq, 768]
    col_max = emb.reshape(-1, emb.shape[-1]).abs().max(dim=0)[0]
    n_out = (col_max > 6.0).sum().item()
    print(f"Outlier dims in 1st-layer input (θ=6.0): {n_out} / {emb.shape[-1]}")
    print("(GPT-2 is small — systematic outliers only emerge above ~6.7B params)\n")

outlier_ppl = perplexity(model_q2, inputs)

# ── Summary ───────────────────────────────────────────────────────────────────
print(f"{'Method':<40} {'PPL':>6}  {'Δ PPL':>7}")
print("-" * 56)
print(f"{'Baseline (FP32)':<40} {baseline_ppl:>6.2f}  {'—':>7}")
print(f"{'Vector-wise INT8 (no outlier sep.)':<40} {quant_ppl:>6.2f}  {quant_ppl - baseline_ppl:>+7.2f}")
print(f"{'Full LLM.int8() (outlier-aware)':<40} {outlier_ppl:>6.2f}  {outlier_ppl - baseline_ppl:>+7.2f}")
